In [1]:
import pandas as pd
import numpy as np

import math

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
from statsmodels.formula.api import ols, glm
import statsmodels.api as sm

from patsy import dmatrix

crab = pd.read_csv("../../data/crab.csv")
salary = pd.read_csv('../../data/salary.csv')
wells = pd.read_csv("../../data/wells.csv")

## Fit a multivariable logistic regression
### Using the knowledge gained in the video you will revisit the crab dataset to fit a multivariate logistic regression model. In chapter 2 you have fitted a logistic regression with width as explanatory variable. In this exercise you will analyze the effects of adding color as additional variable.

### The color variable has a natural ordering from medium light, medium, medium dark and dark. As such color is an ordinal variable which in this example you will treat as a quantitative variable.

### The crab dataset is preloaded in the workspace. Also note that the only difference in the code from the univariate case is in the formula argument, where now you will add structure to incorporate the new variable.

### Instructions
-    Import necessary functions from statsmodels library for GLMs.
-    Define formula argument where width and color are explanatory variables and y is the response.
-    Fit a multivariate logistic regression model using glm() function.
-    Print model results.

In [2]:
# Import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import glm

# Define model formula
formula = 'y ~ width + color'

# Fit GLM
model = glm(formula, data = crab, family = sm.families.Binomial()).fit()

# Print model summary
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      170
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -94.561
Date:                Sat, 16 Mar 2024   Deviance:                       189.12
Time:                        03:49:24   Pearson chi2:                     170.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1909
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.0708      2.807     -3.588      0.0

## The effect of multicollinearity
### Using the crab dataset you will analyze the effects of multicollinearity. Recall that multicollinearity can have the following effects:

### Coefficient is not significant, but variable is highly correlated with "y".
-    Adding/removing a variable significantly changes coefficients.
-    Not logical sign of the coefficient.
-    Variables have high pairwise correlation.

### Instructions
-    Import necessary functions from statsmodels library for GLMs.
-    Fit a multivariate logistic regression model with weight and width as explanatory variables and y as the response.
-    View model results using print() function.

In [3]:
# Import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import glm

# Define model formula
formula = 'y ~ weight + width'

# Fit GLM
model = glm(formula, data = crab, family = sm.families.Binomial()).fit()

# Print model summary
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      170
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -96.446
Date:                Sat, 16 Mar 2024   Deviance:                       192.89
Time:                        03:51:19   Pearson chi2:                     167.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1730
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.3547      3.528     -2.652      0.0

## Compute VIF
### As you learned in the video one of the most widely used diagnostic for multicollinearity is the variance inflation factor or VIF, which is computed for each explanatory variable.

### Recall from the video that the rule of thumb threshold is VIF at the level of 2.5, meaning if the VIF is above 2.5 you should consider there is effect of multicollinearity on your fitted model.

### The previously fitted model and crab dataset are preloaded in the workspace.

### Instructions
-    From statsmodels import variance_inflation_factor.
-    From crab dataset choose weight, width and color and save as X. Add Intercept column of ones to X.
-    Using pandas function DataFrame() create an empty vif dataframe and add column names of X in column Variables.
-    For each variable compute VIF using the variance_inflation_factor()function and save in vif dataframe with VIF column name.

In [4]:
# Import functions
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Get variables for which to compute VIF and add intercept term
X = crab[["weight", "width", "color"]]
X["Intercept"] = 1

# Compute and view VIF
vif = pd.DataFrame()
vif["variables"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# View results using print
print(vif)

   variables         VIF
0     weight    4.691018
1      width    4.726378
2      color    1.076594
3  Intercept  414.163343


/tmp/ipykernel_364806/25132915.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Intercept"] = 1


## Checking model fit
### In the video you analyzed the example of an improvement in the model fit by adding additional variable on the wells data. Continuing with this data set you will see how further increase in model complexity effects deviance and model fit.

### The dataset wells have been preloaded in the workspace.

### Instructions
-    Fit a logistic regression model with switch as the response and distance100 and arsenic as explanatory variables.
-    Compute the difference in deviance of the intercept only model and the model including all the variables.
-    Print the computed difference.

In [5]:
# Import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import glm

# Define model formula
formula = 'switch ~ distance100 + arsenic'

# Fit GLM
model_dist_ars = glm(formula, data = wells, family = sm.families.Binomial()).fit()

# Compare deviance of null and residual model
diff_deviance = model_dist_ars.null_deviance - model_dist_ars.deviance

# Print the computed difference in deviance
print(diff_deviance)

188.76305963384948


## Compare two models
### From previous exercise you have fitted a model with distance100 and arsenic as explanatory variables. In this exercise you will analyze the impact on the model fit for each of the added variables.

### Recall that the models you fitted are as follows and have been preloaded in the workspace:
-    1.) model_dist = 'switch ~ distance100'
-    2.) model_dist_ars = 'switch ~ distance100 + arsenic

### The dataset wells has also been preloaded in the workspace.

### Instructions
-    Compute the difference in deviance when distance100 is added to the null model.
-    Compute the difference in deviance when arsenic is added to the model with distance100 variable.

In [6]:
model_dist = glm('switch ~ distance100', data = wells, family = sm.families.Binomial()).fit()
model_dist_ars = glm('switch ~ distance100 + arsenic', data = wells, family = sm.families.Binomial()).fit()

In [7]:
# Compute the difference in adding distance100 variable
diff_deviance_distance = model_dist.null_deviance - model_dist.deviance

# Print the computed difference in deviance
print('Adding distance100 to the null model reduces deviance by: ', 
      round(diff_deviance_distance,3))

# Compute the difference in adding arsenic variable
diff_deviance_arsenic = model_dist.deviance - model_dist_ars.deviance

# Print the computed difference in deviance
print('Adding arsenic to the distance model reduced deviance further by: ', 
      round(diff_deviance_arsenic,3))

Adding distance100 to the null model reduces deviance by:  42.726
Adding arsenic to the distance model reduced deviance further by:  146.037


## Deviance and linear transformation
### As you have seen in previous exercises the deviance decreased as you added a variable that improves the model fit. In this exercise you will consider the well switch data example and the model you fitted with distance variable, but you will assess what happens when there is a linear transformation of the variable.

### Note that the variable distance100 is the original variable distance divided by 100 to make for more meaningful representation and interpretation of the results. You can inspect the data with wells.head() to view the first 5 rows of data.

### The wells dataset and the model 'swicth ~ distance100' has been preloaded as model_dist.

### Instructions
-    Import statsmodels as sm and the glm() function.
-    Fit a logistic regression model with distance as the explanatory variable and switch as the response and save as model_dist_1.
-    Check and print the difference in deviance of the current model and the model with distance100 as the explanatory variable.

In [ ]:
# Import functions
import statsmodels.api as sm
from statsmodels.formula.api import glm

# Fit logistic regression model as save as model_dist_1
model_dist_1 = glm('switch ~ distance', data = wells, family = sm.families.Binomial()).fit()

# Check the difference in deviance of model_dist_1 and model_dist
print('Difference in deviance is: ', round(model_dist_1.deviance - model_dist.deviance,3))

## Model matrix for continuous variables
### In the video you learned about the model formula, the under-the-hood workings of the dmatrix() to obtain the model matrix and how it relates to the glm() function. As you have learned the input to dmatrix() is the right hand side of the glm() formula argument. In case the variables are part of the dataframe, then you should also specify the data source via the data argument.  <code>dmatrix('y ~ x1 + x2',  data = my_data)</code>

### In this exercise you will analyze and confirm the structure of your model before model fit.

### The dataset wells has been preloaded in the workspace.

### Instructions 1/2
-    Import dmatrix from patsy, use it to construct the model_matrix with 'arsenic', and print the first 5 rows.

In [9]:
# Import function dmatrix()
from patsy import dmatrix

# Construct model matrix with arsenic
model_matrix = dmatrix('arsenic', data = wells, return_type = 'dataframe')
print(model_matrix.head())

   Intercept  arsenic
0        1.0     2.36
1        1.0     0.71
2        1.0     2.07
3        1.0     1.15
4        1.0     1.10


-    Import dmatrix from patsy, construct the model_matrix with 'arsenic' and 'distance100', and print the first 5 rows.

In [10]:
# Import function dmatrix()
from patsy import dmatrix

# Construct model matrix with arsenic and distance100
model_matrix = dmatrix('arsenic + distance100', data = wells, return_type = 'dataframe')
print(model_matrix.head())

   Intercept  arsenic  distance100
0        1.0     2.36      0.16826
1        1.0     0.71      0.47322
2        1.0     2.07      0.20967
3        1.0     1.15      0.21486
4        1.0     1.10      0.40874


## Variable transformation
### Continuing with the wells you will practice applying variable transformation directly in the formula and model matrix setting without the need to add the transformed data to the data frame first. You will also revisit the computation of model error or deviance to see if the transformation improved the model fit.

### Recall the structure of dmatrix() function is the right hand side of the glm() formula argument in addition to the data argument.  <code>dmatrix('y ~ x1 + x2', data = my_data)</code>

### The dataset wells and the model model_ars with arsenic (original variable) have been preloaded in the workspace.

### Instructions 1/3
-    Import numpy as np, and dmatrix from patsy.
-    Construct a model matrix by applying the logarithm transformation on arsenic using numpy log()

In [11]:
# Import function dmatrix
import numpy as np
from patsy import dmatrix

# Construct model matrix for arsenic with log transformation
dmatrix('np.log(arsenic)', data = wells,
       return_type = 'dataframe').head()

,Intercept,np.log(arsenic)
0,1.0,0.858662
1,1.0,-0.342490
2,1.0,0.727549
3,1.0,0.139762
4,1.0,0.095310


-    Import statsmodels.api as sm, glmfrom statsmodels.formula.api and numpyas np.
-    Fit and print summary of a glm model with the response switch and log transformed arsenic explanatory variable defining the transformation in the formula.

In [12]:
# Import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import glm
import numpy as np

# Define model formula
formula = 'switch ~ np.log(arsenic)'

# Fit GLM
model_log_ars = glm(formula, data = wells, 
                     family = sm.families.Binomial()).fit()

# Print model summary
print(model_log_ars.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 switch   No. Observations:                 3010
Model:                            GLM   Df Residuals:                     3008
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1987.6
Date:                Sun, 17 Mar 2024   Deviance:                       3975.3
Time:                        03:55:17   Pearson chi2:                 3.01e+03
No. Iterations:                     4   Pseudo R-squ. (CS):            0.04187
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0966      0.041     

## Coding categorical variables
### In previous exercises you practiced creating model matrices for continuous variables and applying variable transformation. During this exercise you will practice the ways of coding a categorical variable.

### Categorical data provide a way to analyze and compare relationships given different groups or factors. Hence, choosing a reference group is important and often, depending on the study at hand, you might want to change the reference group, from the default one. One frequently used reason for changing the reference group is that the interpretation of coefficient estimates is more applicable and interesting given the study.

### For this exercise you will revisit the crab dataset where colorand spine are categorical variables.

### The dataset crab is preloaded in the workspace.

### Instructions 1/2
-    Import dmatrix from patsy.
-    Using dmatrix() construct and print a model matrix with color as categorical variable using C()function.

In [14]:
# Import function dmatrix
from patsy import dmatrix

# Construct and print model matrix for color as categorical variable
print(dmatrix('C(color)', data = crab,
     	   return_type = 'dataframe').head())

   Intercept  C(color)[T.2]  C(color)[T.3]  C(color)[T.4]
0        1.0            1.0            0.0            0.0
1        1.0            0.0            1.0            0.0
2        1.0            0.0            0.0            0.0
3        1.0            0.0            1.0            0.0
4        1.0            0.0            1.0            0.0


-    Change the reference group to be the medium dark, i.e. group number 3 using the Treatment argument.

In [15]:
# Import function dmatrix
from patsy import dmatrix

# Construct and print the model matrix for color with reference group 3
print(dmatrix('C(color, Treatment(3))', 
     	  data = crab,
     	  return_type = 'dataframe').head())

   Intercept  C(color, Treatment(3))[T.1]  C(color, Treatment(3))[T.2]  \
0        1.0                          0.0                          1.0   
1        1.0                          0.0                          0.0   
2        1.0                          1.0                          0.0   
3        1.0                          0.0                          0.0   
4        1.0                          0.0                          0.0   

   C(color, Treatment(3))[T.4]  
0                          0.0  
1                          0.0  
2                          0.0  
3                          0.0  
4                          0.0  


## Modeling with categorical variable
### In previous exercises you have fitted a logistic regression model with color as explanatory variable along with width where you treated the color as quantitative variable. In this exercise you will treat color as a categorical variable which when you construct the model matrix will encode the color into 3 variables with 0/1 encoding.

### Recall that the default encoding in dmatrix() uses the first group as a reference group. To view model matrix as a dataframe an additional argument in dmatrix(), namely, return_type will be set to 'dataframe'.

### The color variable has a natural ordering as follows:
-    1: medium light
-    2: medium
-    3: medium dark
-    4: dark

### The crab dataset is preloaded in the workspace.

### Instructions 1/4
-    Construct a model_matrix with color as a variable. color should be treated as categorical and the reference group set to 4 using Treatment() function.
-    Fit and print the results of a logistic model with y as the response.

In [16]:
# Construct model matrix
model_matrix = dmatrix('C(color, Treatment(4))' , data = crab, 
                       return_type = 'dataframe')

# Print first 5 rows of model matrix dataframe
print(model_matrix.head())

# Fit and print the results of a glm model with the above model matrix configuration
model = glm('y ~ C(color, Treatment(4))', data = crab, 
            family = sm.families.Binomial()).fit()

print(model.summary())

   Intercept  C(color, Treatment(4))[T.1]  C(color, Treatment(4))[T.2]  \
0        1.0                          0.0                          1.0   
1        1.0                          0.0                          0.0   
2        1.0                          1.0                          0.0   
3        1.0                          0.0                          0.0   
4        1.0                          0.0                          0.0   

   C(color, Treatment(4))[T.3]  
0                          0.0  
1                          1.0  
2                          0.0  
3                          1.0  
4                          1.0  
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      169
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   S

-    To the previous model matrix and logistic regression model add width as additional explanatory variable. View model matrix before fitting the model and then view model results.

In [17]:
# Construct model matrix
model_matrix = dmatrix('C(color, Treatment(4)) + width' , data = crab, 
                       return_type = 'dataframe')

# Print first 5 rows of model matrix
print(model_matrix.head())

# Fit and print the results of a glm model with the above model matrix configuration
model = glm('y ~ C(color, Treatment(4)) + width', data = crab, 
            family = sm.families.Binomial()).fit()

print(model.summary())

   Intercept  C(color, Treatment(4))[T.1]  C(color, Treatment(4))[T.2]  \
0        1.0                          0.0                          1.0   
1        1.0                          0.0                          0.0   
2        1.0                          1.0                          0.0   
3        1.0                          0.0                          0.0   
4        1.0                          0.0                          0.0   

   C(color, Treatment(4))[T.3]  width  
0                          0.0   28.3  
1                          1.0   22.5  
2                          0.0   26.0  
3                          1.0   24.8  
4                          1.0   26.0  
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      168
Model Family:                Binomial   Df Model:                            4

## Interaction terms
### In the video you learned how to include interactions in the model structure when there is one continuous and one categorical variable. In this exercise you will analyze the effects of interaction between two continuous variables.

### You will use centered variables instead of original values to be able to interpret the coefficient effects more easily, i.e. from the level of the mean values rather than 0 which may not be logical for the study at hand. In other words we don't want to interpret the model by assuming 0 for arsenic or distance100 variables.

### The model 'switch ~ distance100 + arsenic' has been preloaded as model_dist_ars in the workspace.
### Also wells dataset is preloaded.

### Instructions 1/2
-    Fit a logistic model with switch as the response and centered distance100,arsenic and the interaction term between distance100 and arsenic as explanatory variables. Use center() to center the variables.
Print the model summary.

In [18]:
# Import libraries
import statsmodels.api as sm
from statsmodels.formula.api import glm

# Fit GLM and print model summary
model_int = glm('switch ~ center(distance100) + center(arsenic) + center(distance100):center(arsenic)', 
                data = wells, family = sm.families.Binomial()).fit()

# View model results
print(model_int.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 switch   No. Observations:                 3010
Model:                            GLM   Df Residuals:                     3006
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1956.2
Date:                Sun, 17 Mar 2024   Deviance:                       3912.4
Time:                        04:23:43   Pearson chi2:                 3.08e+03
No. Iterations:                     4   Pseudo R-squ. (CS):            0.06168
Covariance Type:            nonrobust                                         
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In